In [1]:
]activate proxgrad

 Activating environment at `C:\Users\darre\Documents\_cornell 20-21\orie 4741\dream-team\proxgrad models\proxgrad\Project.toml`


In [2]:
using DataFrames, LowRankModels, Plots, CSV, Random

In [3]:
df = CSV.read("../data collection/weekly_ff_elo.csv", DataFrame, missingstring = "NA")

,week,Player,Pos,team1,PassingYds,PassingTD,Int,PassingAtt
,Int64,String,String?,String,Int64,Int64,Int64,Int64
1,1,Tony Romo,QB,DAL,353,3,0,27
2,1,Drew Brees,QB,NOR,358,6,1,34
3,1,Brodie Croyle,QB,KAN,177,2,0,24
4,1,Trent Edwards,QB,BUF,212,2,0,25
5,1,Bruce Gradkowski,QB,OAK,17,0,0,2
6,1,Kyle Orton,QB,DEN,243,1,0,28
7,1,Matt Ryan,QB,ATL,229,2,0,36
8,1,Tom Brady,QB,NWE,378,2,1,53
9,1,Matt Hasselbeck,QB,SEA,279,3,2,36


In [21]:
Random.seed!(0)

df = df[.!(ismissing.(df[!, :StandardFantasyPoints])), :]
df = df[shuffle(1:end), :]

train_proportion = 0.8
n = size(df, 1)
println("Size of dataset: ", string(n))

ntrain = convert(Int, round(train_proportion*n))

target = df[:, [:StandardFantasyPoints, :PPRFantasyPoints, :HalfPPRFantasyPoints]]

train_x = df[1:ntrain, Not([:StandardFantasyPoints, :PPRFantasyPoints, :HalfPPRFantasyPoints])]
test_x = df[ntrain:end, Not([:StandardFantasyPoints, :PPRFantasyPoints, :HalfPPRFantasyPoints])]

train_std = target[1:ntrain, :StandardFantasyPoints]
test_std = target[ntrain:end, :StandardFantasyPoints]

train_ppr = target[1:ntrain, :PPRFantasyPoints]
test_ppr = target[ntrain:end, :PPRFantasyPoints]

train_half = target[1:ntrain, :HalfPPRFantasyPoints]
test_half = target[ntrain:end, :HalfPPRFantasyPoints]

# let's take a look
train_x;

Size of dataset: 52814


In [22]:
labels_real = [
    :PassingYds_cum,
    :PassingTD_cum,
    :Int_cum,
    :PassingAtt_cum,
    :Cmp_cum,
    :RushingAtt_cum,
    :RushingYds_cum,
    :RushingTD_cum,
    :Rec_cum,
    :Tgt_cum,
    :ReceivingYds_cum,
    :ReceivingTD_cum,
    :FL_cum,
    :team1_score_cum,
    :team2_score_cum,
    :PassingYds_prev,
    :PassingTD_prev,
    :Int_prev,
    :PassingAtt_prev,
    :Cmp_prev,
    :RushingAtt_prev,
    :RushingYds_prev,
    :RushingTD_prev,
    :Rec_prev,
    :Tgt_prev,
    :ReceivingYds_prev,
    :ReceivingTD_prev,
    :FL_prev,
    :team1_score_prev,
    :team2_score_prev,
    :elo1_pre,
    :elo2_pre
]

train_real = train_x[:, labels_real]
test_real = test_x[:, labels_real];
df_real = df[:, labels_real];

In [6]:
A = zeros(42251, 5)*zeros(5, 32)
loss = QuadLoss()
reg = QuadReg(0.01)
k = 5
Ω = observations(train_real)
glrm = GLRM(A, loss, reg, reg, k, obs = Ω);

In [35]:
Random.seed!(0)
init_svd!(glrm)
X,W,ch = fit!(glrm, max_iters = 1000, scale = false);

Fitting GLRM
Iteration 10: objective value = 4.906729234661519e6
Iteration 20: objective value = 1.0022951420242077e6
Iteration 30: objective value = 65707.60416848093
Iteration 40: objective value = 7958.754605960012
Iteration 50: objective value = 1735.302526011949
Iteration 60: objective value = 439.5780696719361


In [34]:
ch

ConvergenceHistory("ProxGradGLRM", [2.539209668890844e7, 2.225569191136907e7, 1.9381649016092658e7, 1.676566924294408e7, 1.4401334667321317e7, 1.2280172572714843e7, 1.03918275916812e7, 8.724276843586938e6, 7.26408271229118e6, 5.99667611773635e6  …  748.1384909337007, 629.0102267260863, 547.4709134890588, 515.1920562091717, 439.5780696719363, 386.553680020784, 365.8657723603099, 318.9693272806711, 286.6718600768849, 273.73507601864145], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 33.335999965667725, 38.13199996948242, 43.651999950408936, 52.77199983596802, 61.978999853134155, 71.4449999332428, 81.16599988937378, 86.77899980545044, 93.35699987411499  …  424.64299964904785, 431.18499970436096, 439.2669997215271, 447.1039996147156, 453.6639997959137, 458.9249997138977, 468.5259997844696, 477.62799978256226, 48

In [36]:
B = impute_missing(glrm)

42251×32 Array{Number,2}:
  0.298198  -1.66978    -0.0915753   …  -2.37458     1.80884   -1.46682
  2.23011    1.43722     1.71714         1.5464     -0.382431  -2.68269
  4.67355    1.19756     2.94718         0.106353   -3.3549    -5.29859
 -1.24312    1.47656     1.18151        -3.9113      0.996      3.18023
 -0.393378   0.701416   -0.826804        3.63716    -2.31072    0.713436
 -0.279185  -0.412313   -0.206634    …   0.459417    3.27338   -0.609738
 -0.88629   -2.9881     -2.50101         3.71287    -1.8319    -4.10687
 -4.13406    1.13208    -1.41463         0.245825   -1.17473    5.31428
 -3.13257   -3.78719    -2.26242        -2.70117     6.89957   -0.637992
 -2.02739    1.93701     0.00348299      0.368716   -0.207426   3.71993
  0.612656  -0.736755    0.390903    …  -1.60343     3.09369   -0.991855
  1.38515    0.0201841   0.735599       -1.74446    -0.894655   0.0546731
  2.07162   -6.55816    -0.62237        -6.85893     2.96481   -7.41308
  ⋮                             

In [71]:
w = B \ train_std

32-element Array{Float64,1}:
   17.000871134610364
  -22.842484729950087
   -8.621782974642311
  -59.58383601818882
  -19.958118988618555
   14.146435371164268
   70.59274110073089
  -41.16728558735958
  -20.403052068780664
   12.540949531421521
    9.254074974099602
   20.48879527585912
    0.6859721083316462
    ⋮
   19.28233879144565
    7.823661292783758
    7.706914340232507
  -14.825210511590784
   -0.33848491358999006
    0.6700547477632587
  -48.52680793298255
  -23.857905656371294
  -38.40667767059051
  -81.8571677172422
   96.37958449185565
 -351.2150162069245

In [73]:
using LinearAlgebra

In [ ]:
sum(w'.*Matrix(test_real), dims = 2)

The matrix imputation method seems to be misapplied or misscaled in some way such that the scalings do not work to predict points.